# Data Collection

In [16]:
import psaw
from psaw import PushshiftAPI
import datetime as dt
api = PushshiftAPI()
from pprint import pprint
import pickle

In [10]:
# Function to go through each day and collect comments on r/bitcoin
def day_by_day(year, month, start_date, end_date, l, count):
    for j in range(start_date, end_date):
        start_epoch=int(dt.datetime(year, month, j).timestamp())
        end_epoch=int(dt.datetime(year, month, j+1).timestamp())

        submissions_full = api.search_submissions(after=start_epoch,
                                before=end_epoch,
                                subreddit="Bitcoin")

        for i in submissions_full:
            try:
                if i.selftext != "" and i.selftext != "[deleted]" and i.selftext != "[removed]":
                    l.append((i.d_))
                    count += 1
            except AttributeError:
                pass
    return (l, count)

In [14]:
days = [[31, 28, 31], [30, 31, 30], [31, 31, 30], [31, 30, 31]]
days_leap = [[31, 29, 31], [30, 31, 30], [31, 31, 30], [31, 30, 31]]
months = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
years = [2014, 2015, 2016, 2017, 2018]

In [15]:
# Get bitcoin posts
my_dict = {}
for year in years:
    for i in range(len(months)):
            l = []
            count = 0
            for j in range(3):
                (l, count) = day_by_day(year, months[i][j], 1, days[i][j], l, count)
            quarter = "Q" + str(i+1) + str(year)
            my_dict[quarter] = (l, count)
            print("In Q", str(i+1), str(year), ". Count is ", str(count))

In Q 1 2014 . Count is  12733
In Q 2 2014 . Count is  7682
In Q 3 2014 . Count is  6436
In Q 4 2014 . Count is  7432
In Q 1 2015 . Count is  8582
In Q 2 2015 . Count is  6265
In Q 3 2015 . Count is  5562
In Q 4 2015 . Count is  4834
In Q 1 2016 . Count is  2737
In Q 2 2016 . Count is  3070
In Q 3 2016 . Count is  2835
In Q 4 2016 . Count is  3358
In Q 1 2017 . Count is  8461
In Q 2 2017 . Count is  10125
In Q 3 2017 . Count is  14363
In Q 4 2017 . Count is  36642
In Q 1 2018 . Count is  9944
In Q 2 2018 . Count is  6833
In Q 3 2018 . Count is  6563
In Q 4 2018 . Count is  6797


In [17]:
# Write to file for later use
with open('ALL_Bitcoin_Post.txt', 'wb') as f:
    pickle.dump(my_dict, f)

In [18]:
# Function to go through each day and collect comments on r/ethereum
def day_by_day_ether(year, month, start_date, end_date, l, count):
    for j in range(start_date, end_date):
        start_epoch=int(dt.datetime(year, month, j).timestamp())
        end_epoch=int(dt.datetime(year, month, j+1).timestamp())

        submissions_full = api.search_submissions(after=start_epoch,
                                before=end_epoch,
                                subreddit="ethereum")

        for i in submissions_full:
            try:
                if i.selftext != "" and i.selftext != "[deleted]" and i.selftext != "[removed]":
                    l.append((i.d_))
                    count += 1
            except AttributeError:
                pass
    return (l, count)

In [20]:
# Get ethereum posts
my_dict_ether = {}
for year in years:
    for i in range(len(months)):
            l = []
            count = 0
            for j in range(3):
                (l, count) = day_by_day_ether(year, months[i][j], 1, days[i][j], l, count)
            quarter = "Q" + str(i+1) + str(year)
            my_dict_ether[quarter] = (l, count)
            print("In Q", str(i+1), str(year), ". Count is ", str(count))

In Q 1 2014 . Count is  195
In Q 2 2014 . Count is  120
In Q 3 2014 . Count is  215
In Q 4 2014 . Count is  103
In Q 1 2015 . Count is  285
In Q 2 2015 . Count is  366
In Q 3 2015 . Count is  1439
In Q 4 2015 . Count is  550
In Q 1 2016 . Count is  1681
In Q 2 2016 . Count is  2073
In Q 3 2016 . Count is  1926
In Q 4 2016 . Count is  1157
In Q 1 2017 . Count is  1779
In Q 2 2017 . Count is  3384
In Q 3 2017 . Count is  1937
In Q 4 2017 . Count is  2662
In Q 1 2018 . Count is  1866
In Q 2 2018 . Count is  1054
In Q 3 2018 . Count is  1353
In Q 4 2018 . Count is  869


In [21]:
# Write to file for later use
with open('ALL_Ethereum_Post.txt', 'wb') as f2:
    pickle.dump(my_dict_ether, f2)

# Topic Modeling

In [22]:
# imports
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bethanybailey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Rename
my_dict_bit = my_dict

In [264]:
# Extend stop words to include frequent link info
stop_words = stopwords.words('english')
stop_words.extend(['http', 'www', 'com', 'www_reddit', 'reddit', 'edu'])

In [265]:
# Taking out links, taking only posts greater than 500
def find_long_posts(data):
    long_post_text_dict = {}
    for quarter, data in data.items():
        new_count = 0
        new_l = []
        for post_dict in data[0]:
            if len(re.sub(r'http\S+', '', post_dict["selftext"])) > 100:
                new_count += 1
                new_l.append(re.sub(r'http\S+', '', post_dict["selftext"]))
        long_post_text_dict[quarter] = (new_l, new_count)
    return long_post_text_dict

In [266]:
bit_text_long_dict = find_long_posts(my_dict_bit)

In [267]:
ether_text_long_dict = find_long_posts(my_dict_ether)

In [268]:
# Checking
for i in bit_text_long_dict['Q12016'][0]:
    print(i)

There are some great points in the documentary now that make you go wow.

Charlie Shrem saying "I'm spending thousands on lawyers. I don't want to go to jail", this was before his Silk Road involvement.

Daniel Mross paid 2700 bitcoins for 25 Ghs to Butterfly labs in 2012 (that's over 1M today), back then it was $13.5k.

Plus all the other goodies in the documentary... Makes you forget about the blocksize debates and all the other junk at the moment.
###Status of the Bitcoin network as of Saturday, January 02, 2016 at 00:00:03 EST:

**Total bitcoins:** 15,033,489.8023

**Height:** 391,342

**Difficulty:** 1.03880340815e+11

######Statistics for the past 24 hours:

**Number of blocks mined:** 132

**Total bitcoins output (amount sent):** 895,846.210875

**Total fees:** 19.50254948

**Average time until block found:** 10 minutes, 54 seconds

**Estimated hashrate:** 681,637,406.657 gh/s

**Current price:** US$434.58

*Data provided by [Smartbit.com.au](

***

^^I ^^am ^^a ^^bot. **[^^My ^


 Satoshi predicted big mining farms in the early days, than he would talk about the "longest chain" when asked about consensus. What is the longest chain? It is nothing more than the one most people are using, right now it is the chain produced by Bitcoin Core client. Even if most miners do not switch to another client at first, they will soon have to switch after most users 'cause the previous chain will quickly degrade in usage and value until it dies and we get a new longer chain.

When you are using Bitcoin Core or an alternative client of the same chain you are giving one vote to it, if you instead start running and using Bitcoin Classic or something else that's where your CPU vote is going to.
So now it's out in the open.  Mike's announcement was a planned and scheduled event timed to create the most damage at just the right time.  



"I don't know what time it is, it will be in the New York Times any minute saying Mike Hearn broke up with Bitcoin and called it a failed experim

As the title states. I study economics so I am very curious as to how big miners plan for the halving of income.
I found this link...



Looks like the US gov. is paving the way to BTC.
I always knew they where working to get this off the ground.  I can see some serious adoption comming fast!

it may not be Dogecoin, but I still say... THE MOON' Yeah!!!
I see them sold on eBay but the payout in bitcoin is equal too the value of the contract usually. Are they hoping for a price difference, laundering money, not aure why this is even a service except from the seller being paid for his equipment time
The recently added opt-in ReplaceByFee feature uses the nSequence number as a flag to indicate if the transaction is subject to RBF or not (or well, intended to be, in the end it's still up to individual nodes and miners to apply their own replacement policy).

However, in a Bitcoin transaction, **each** input has its own nSequence. 

What nSequence is supposed to be interpreted as the RBF fl

In [269]:
print("Count of Bitcoin posts over 100 characters without links:")
for key, value in bit_text_long_dict.items():
    print(key, ":", value[1])

Count of Bitcoin posts over 100 characters without links:
Q12014 : 10821
Q22014 : 6483
Q32014 : 5523
Q42014 : 6321
Q12015 : 7334
Q22015 : 5466
Q32015 : 4767
Q42015 : 4132
Q12016 : 2355
Q22016 : 2585
Q32016 : 2428
Q42016 : 2861
Q12017 : 7051
Q22017 : 8414
Q32017 : 11746
Q42017 : 29662
Q12018 : 8282
Q22018 : 5366
Q32018 : 5027
Q42018 : 5268


In [270]:
print("Count of Ethereum posts over 100 characters without links:")
for key, value in ether_text_long_dict.items():
    print(key, ":", value[1])

Count of Ethereum posts over 100 characters without links:
Q12014 : 172
Q22014 : 108
Q32014 : 186
Q42014 : 90
Q12015 : 256
Q22015 : 321
Q32015 : 1224
Q42015 : 481
Q12016 : 1484
Q22016 : 1833
Q32016 : 1675
Q42016 : 1012
Q12017 : 1583
Q22017 : 3028
Q32017 : 1732
Q42017 : 2335
Q12018 : 1702
Q22018 : 917
Q32018 : 1109
Q42018 : 732


In [271]:
# Remove new line characters
for key, value in bit_text_long_dict.items():
    new_tup = ([re.sub('\s+', ' ', sent) for sent in value[0]], value[1])
    bit_text_long_dict[key] = new_tup

for key, value in ether_text_long_dict.items():
    new_tup = ([re.sub('\s+', ' ', sent) for sent in value[0]], value[1])
    ether_text_long_dict[key] = new_tup

In [272]:
# Remove distracting single quotes
for key, value in bit_text_long_dict.items():
    new_tup = ([re.sub("\'", "", sent) for sent in value[0]], value[1])
    bit_text_long_dict[key] = new_tup

for key, value in ether_text_long_dict.items():
    new_tup = ([re.sub("\'", "", sent) for sent in value[0]], value[1])
    ether_text_long_dict[key] = new_tup

In [273]:
# Remove '*'
for key, value in bit_text_long_dict.items():
    new_tup = ([re.sub("\*", "", sent) for sent in value[0]], value[1])
    bit_text_long_dict[key] = new_tup

for key, value in ether_text_long_dict.items():
    new_tup = ([re.sub("\*", "", sent) for sent in value[0]], value[1])
    ether_text_long_dict[key] = new_tup

In [274]:
# Remove '^'
for key, value in bit_text_long_dict.items():
    new_tup = ([re.sub("\^", "", sent) for sent in value[0]], value[1])
    bit_text_long_dict[key] = new_tup

for key, value in ether_text_long_dict.items():
    new_tup = ([re.sub("\^", "", sent) for sent in value[0]], value[1])
    ether_text_long_dict[key] = new_tup

In [275]:
# Remove '#'
for key, value in bit_text_long_dict.items():
    new_tup = ([re.sub("\#", "", sent) for sent in value[0]], value[1])
    bit_text_long_dict[key] = new_tup

for key, value in ether_text_long_dict.items():
    new_tup = ([re.sub("\#", "", sent) for sent in value[0]], value[1])
    ether_text_long_dict[key] = new_tup

In [276]:
# Remove '|'
for key, value in bit_text_long_dict.items():
    new_tup = ([re.sub("\|", "", sent) for sent in value[0]], value[1])
    bit_text_long_dict[key] = new_tup

for key, value in ether_text_long_dict.items():
    new_tup = ([re.sub("\|", "", sent) for sent in value[0]], value[1])
    ether_text_long_dict[key] = new_tup

In [277]:
# Check cleaning
for i in bit_text_long_dict['Q12016'][0]:
    print(i)

There are some great points in the documentary now that make you go wow. Charlie Shrem saying "Im spending thousands on lawyers. I dont want to go to jail", this was before his Silk Road involvement. Daniel Mross paid 2700 bitcoins for 25 Ghs to Butterfly labs in 2012 (thats over 1M today), back then it was $13.5k. Plus all the other goodies in the documentary... Makes you forget about the blocksize debates and all the other junk at the moment.
Status of the Bitcoin network as of Saturday, January 02, 2016 at 00:00:03 EST: Total bitcoins: 15,033,489.8023 Height: 391,342 Difficulty: 1.03880340815e+11 Statistics for the past 24 hours: Number of blocks mined: 132 Total bitcoins output (amount sent): 895,846.210875 Total fees: 19.50254948 Average time until block found: 10 minutes, 54 seconds Estimated hashrate: 681,637,406.657 gh/s Current price: US$434.58 Data provided by [Smartbit.com.au](  I am a bot. [My commands](  /r/crypto_bot  [Message my creator](  [Source code](
First of all the

I got in yesterday and everytime i buy,, it goes down within the hour, it alway happens. Im sorry. edit: i meant i was back in yesterday, im not a noob, i have been playing with it for a while, and i had this experience a couple of times, you buy and it goes down almost instantly..lol and i know the drill: HODL and to the moon! Sorry for the confusion.
Three weeks ago I submitted x4 $25 Starbucks and I get a response today that two cards were disable and I got paid for the other two, I checked the two that were claimed to be disable, but are fine. Though one of them are now registered to an account, not sure whos. Has anyone had this problem with them before?
Here I will summarize the status quo, diagnose the current state of Bitcoin and propose a future roadmap, all in layman terms. Lets first tackle this crisis and move on: We all know that this crash was triggered by Mike Hearn. His childish tantrum-esque departure was given undue amount of attention. A lot of deserved praise has be

Hello r/bitcoin, Im in a bit of a pickle. Ive sent a few BTC (a lot of money) to my Arcbit wallet, more than I have sent before (usually 1 BTC max) and now when I open the app it insta-crashes. Ive rebooted many times and tried opening the app plenty of times. My iphone is on iOS 8.4 jailbreak. Does anybody know where the private key would be found for my Arcbit wallet in the iOS file structure? Ive got iFile and dont want to risk reinstalling the application for Arcbit in case it destroys the wallet seed/private keys. Ironically I was going to try the in app backup option today but its too late for that. Suddenly the app started crashing. Please help me save this money! Thank you bitcoiners Edit: Good news! The app was crashing with wi-fi enabled. Went to Settings -> Advanced Options -> Recoverable switched On. Deleted app and reinstalled from iTunes. This only worked because I was using the native ArcBit wallet so the iOS keychain had my backup passphrase backed up. Now app does not 

In [278]:
# Simple preprocess, which will remove punctuation, tokenize, and put the sentence into a word list
def sent_to_words(sentence_dict):
    for key, value in sentence_dict.items():
        new_tup = ([gensim.utils.simple_preprocess(str(sent), deacc=True) for sent in value[0]], value[1])
        sentence_dict[key] = new_tup

In [279]:
sent_to_words(bit_text_long_dict)

In [280]:
sent_to_words(ether_text_long_dict)

In [327]:
early_years = ['Q32015', 'Q42015', 'Q12016', 'Q22016', 'Q32016', 'Q42016']
late_years = ['Q12017', 'Q22017', 'Q32017', 'Q42017', 'Q12018', 'Q22018', 'Q32018', 'Q42018']
def sep_into_two(sentence_dict):
    this_dict = {}
    early = []
    early_count = 0
    late = []
    late_count = 0
    for key, value in sentence_dict.items():
        if key in early_years:
            for i in value[0]:
                early.append(i)
            early_count += value[1]
        if key in late_years:
            for i in value[0]:
                late.append(i)
            late_count += value[1]
    this_dict['Q32015-Q42016'] = (early, early_count)
    this_dict['Q12017-Q42018'] = (late, late_count)
    return this_dict

In [328]:
bit_two = sep_into_two(bit_text_long_dict)

In [329]:
ether_two = sep_into_two(ether_text_long_dict)

In [330]:
# Find info on size of data after preprocessing
def find_nums(sentence_dict):
    num_token = 0
    num_char = 0
    for key, value in sentence_dict.items():
        for i in value[0]:
            for j in i:
                num_token += 1
                num_char += len(j)
        print(key, "& ", value[1], "& ", num_token, "& ", num_char, "& ", round(num_token/value[1], 2), "\\\\")

In [331]:
find_nums(bit_text_long_dict)
print()
find_nums(ether_text_long_dict)

Q12014 &  10821 &  1306792 &  6079258 &  120.76 \\
Q22014 &  6483 &  2097847 &  9774595 &  323.59 \\
Q32014 &  5523 &  2764014 &  12876445 &  500.46 \\
Q42014 &  6321 &  3506264 &  16327154 &  554.7 \\
Q12015 &  7334 &  4331250 &  20162670 &  590.57 \\
Q22015 &  5466 &  5038423 &  23477476 &  921.78 \\
Q32015 &  4767 &  5615105 &  26178125 &  1177.91 \\
Q42015 &  4132 &  6077824 &  28335991 &  1470.92 \\
Q12016 &  2355 &  6368857 &  29708830 &  2704.4 \\
Q22016 &  2585 &  6656204 &  31042920 &  2574.93 \\
Q32016 &  2428 &  6912343 &  32226628 &  2846.93 \\
Q42016 &  2861 &  7206319 &  33587149 &  2518.81 \\
Q12017 &  7051 &  7942600 &  36992122 &  1126.45 \\
Q22017 &  8414 &  8765253 &  40762205 &  1041.75 \\
Q32017 &  11746 &  9884963 &  45882936 &  841.56 \\
Q42017 &  29662 &  12528833 &  57825135 &  422.39 \\
Q12018 &  8282 &  13376048 &  61747881 &  1615.07 \\
Q22018 &  5366 &  13959050 &  64491116 &  2601.39 \\
Q32018 &  5027 &  14503960 &  67066400 &  2885.21 \\
Q42018 &  5268 & 

In [332]:
find_nums(bit_two)
print()
find_nums(ether_two)

Q32015-Q42016 &  19128 &  2167896 &  10109673 &  113.34 \\
Q12017-Q42018 &  80816 &  10074037 &  46434109 &  124.65 \\

Q32015-Q42016 &  7709 &  930062 &  4367778 &  120.65 \\
Q12017-Q42018 &  13138 &  2359031 &  11102099 &  179.56 \\


In [80]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [230]:
# Check cleaning again
bit_text_long_dict['Q12016'][0][0]

['there',
 'are',
 'some',
 'great',
 'points',
 'in',
 'the',
 'documentary',
 'now',
 'that',
 'make',
 'you',
 'go',
 'wow',
 'charlie',
 'shrem',
 'saying',
 'im',
 'spending',
 'thousands',
 'on',
 'lawyers',
 'dont',
 'want',
 'to',
 'go',
 'to',
 'jail',
 'this',
 'was',
 'before',
 'his',
 'silk',
 'road',
 'involvement',
 'daniel',
 'mross',
 'paid',
 'bitcoins',
 'for',
 'ghs',
 'to',
 'butterfly',
 'labs',
 'in',
 'thats',
 'over',
 'today',
 'back',
 'then',
 'it',
 'was',
 'plus',
 'all',
 'the',
 'other',
 'goodies',
 'in',
 'the',
 'documentary',
 'makes',
 'you',
 'forget',
 'about',
 'the',
 'blocksize',
 'debates',
 'and',
 'all',
 'the',
 'other',
 'junk',
 'at',
 'the',
 'moment']

In [233]:
def final_preprocess(sentence_dict):
    for key, value in sentence_dict.items():
        texts_out = []
        # Build the bigram model
        bigram = gensim.models.Phrases(value[0], min_count=5, threshold=100) # higher threshold fewer phrases.
        # Faster way to get a sentence clubbed as a trigram/bigram
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        # Remove stop words
        text = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in value[0]]
        # Bigrams
        bi_text = [bigram_mod[doc] for doc in text]
        # Lemmatization, keeping only noun, adj, vb, adv
        texts_out = []
        for sent in bi_text:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])
        sentence_dict[key] = (texts_out, value[1])

In [234]:
final_preprocess(bit_text_long_dict)
final_preprocess(ether_text_long_dict)

In [333]:
final_preprocess(bit_two)
final_preprocess(ether_two)

In [235]:
# Check final preprocess
bit_text_long_dict['Q12016'][0][0]

['great',
 'point',
 'documentary',
 'make',
 'go',
 'charlie',
 'shrem',
 'say',
 'be',
 'spend',
 'thousand',
 'lawyer',
 'do',
 'not',
 'want',
 'go',
 'jail',
 'silk',
 'road',
 'involvement',
 'daniel',
 'mross',
 'pay',
 'bitcoin',
 'ghs',
 'butterfly',
 'lab',
 'that',
 's',
 'today',
 'back',
 'goody',
 'documentary',
 'make',
 'forget',
 'blocksize',
 'debate',
 'junk',
 'moment']

In [239]:
def id_2_word_dict(sentence_dict):
    for key, value in sentence_dict.items():
        # Create dictionary
        id2word = corpora.Dictionary(value[0])
        # Create corpus
        texts = value[0]
        # Term Document Frequency
        corpus = [id2word.doc2bow(text) for text in texts]
        # reassign tuple
        sentence_dict[key] = (corpus, id2word, texts)

In [240]:
id_2_word_dict(bit_text_long_dict)

In [241]:
id_2_word_dict(ether_text_long_dict)

In [334]:
id_2_word_dict(bit_two)

In [335]:
id_2_word_dict(ether_two)

In [244]:
def create_lda(sentence_dict):
    new_dict = {}
    for key, value in sentence_dict.items():
        lda_model = gensim.models.ldamodel.LdaModel(corpus=value[0],
                                           id2word=value[1],
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        new_dict[key] = lda_model
    return new_dict

In [245]:
bit_lda_dict = create_lda(bit_text_long_dict)

In [250]:
ether_lda_dict = create_lda(ether_text_long_dict)

In [336]:
two_bit_lda_dict = create_lda(bit_two)

In [337]:
two_ether_lda_dict = create_lda(ether_two)

In [257]:
def print_perplexity(lda_dict, sentence_dict):
    for key, value in lda_dict.items():
        # Perplexity: a measure of how good the model is. lower the better.
        print('\nPerplexity of', key, "is:", value.log_perplexity(sentence_dict[key][0]))

In [338]:
print_perplexity(two_bit_lda_dict, bit_two)
print_perplexity(two_ether_lda_dict, ether_two)


Perplexity of Q32015-Q42016 is: -8.26151236319829

Perplexity of Q12017-Q42018 is: -8.056903025356311

Perplexity of Q32015-Q42016 is: -8.317617839388538

Perplexity of Q12017-Q42018 is: -8.237515587323099


In [340]:
two_bit_lda_dict['Q32015-Q42016'].print_topics()

[(0,
  '0.086*"core" + 0.052*"datum" + 0.043*"hardware" + 0.032*"digital" + 0.019*"satoshi" + 0.019*"full_node" + 0.018*"debit_card" + 0.018*"sit" + 0.016*"euro" + 0.016*"hey_guy"'),
 (1,
  '0.028*"people" + 0.022*"price" + 0.012*"may" + 0.012*"value" + 0.012*"money" + 0.011*"time" + 0.011*"currency" + 0.010*"market" + 0.008*"many" + 0.007*"system"'),
 (2,
  '0.089*"wallet" + 0.040*"use" + 0.034*"address" + 0.030*"blockchain" + 0.030*"coin" + 0.023*"send" + 0.011*"create" + 0.011*"show" + 0.011*"info" + 0.010*"time"'),
 (3,
  '0.053*"btc" + 0.040*"buy" + 0.030*"account" + 0.025*"exchange" + 0.023*"coinbase" + 0.021*"bank" + 0.017*"sell" + 0.017*"use" + 0.017*"pay" + 0.017*"money"'),
 (4,
  '0.048*"instant" + 0.038*"sorry" + 0.030*"cent" + 0.030*"application" + 0.028*"scenario" + 0.027*"offline" + 0.025*"bet" + 0.019*"reset" + 0.019*"anonymous" + 0.018*"weekend"'),
 (5,
  '0.039*"block" + 0.025*"network" + 0.024*"miner" + 0.022*"segwit" + 0.020*"increase" + 0.019*"mining" + 0.016*"node"

In [341]:
two_bit_lda_dict['Q12017-Q42018'].print_topics()

[(0,
  '0.037*"not" + 0.036*"would" + 0.035*"get" + 0.029*"btc" + 0.025*"be" + 0.024*"do" + 0.024*"buy" + 0.022*"know" + 0.019*"want" + 0.019*"money"'),
 (1,
  '0.041*"user" + 0.033*"mining" + 0.022*"community" + 0.019*"digital" + 0.019*"token" + 0.018*"power" + 0.018*"reward" + 0.017*"team" + 0.015*"join" + 0.013*"solution"'),
 (2,
  '0.021*"blockchain" + 0.016*"world" + 0.015*"cryptocurrency" + 0.014*"technology" + 0.014*"system" + 0.013*"project" + 0.012*"company" + 0.011*"business" + 0.011*"platform" + 0.011*"base"'),
 (3,
  '0.106*"bitcoin" + 0.023*"people" + 0.021*"time" + 0.019*"crypto" + 0.018*"year" + 0.017*"see" + 0.013*"think" + 0.013*"new" + 0.012*"day" + 0.011*"make"'),
 (4,
  '0.065*"network" + 0.053*"block" + 0.040*"run" + 0.034*"node" + 0.034*"miner" + 0.031*"lightning" + 0.025*"chain" + 0.024*"code" + 0.016*"number" + 0.015*"core"'),
 (5,
  '0.081*"coinbase" + 0.075*"payment" + 0.048*"pay" + 0.047*"accept" + 0.045*"service" + 0.032*"card" + 0.023*"whole" + 0.022*"tax" 

In [342]:
two_ether_lda_dict['Q32015-Q42016'].print_topics()

[(0,
  '0.034*"would" + 0.020*"could" + 0.017*"make" + 0.016*"people" + 0.014*"think" + 0.011*"price" + 0.010*"money" + 0.010*"exchange" + 0.010*"may" + 0.010*"fund"'),
 (1,
  '0.141*"wallet" + 0.091*"account" + 0.069*"send" + 0.053*"address" + 0.043*"transfer" + 0.034*"file" + 0.027*"key" + 0.019*"show" + 0.019*"web" + 0.013*"coinbase"'),
 (2,
  '0.068*"geth" + 0.057*"parity" + 0.054*"sync" + 0.051*"node" + 0.030*"run" + 0.030*"client" + 0.027*"download" + 0.020*"version" + 0.015*"connect" + 0.015*"server"'),
 (3,
  '0.121*"token" + 0.061*"dao" + 0.027*"investor" + 0.017*"split" + 0.016*"consensus" + 0.014*"prove" + 0.013*"email" + 0.012*"standard" + 0.011*"member" + 0.011*"rule"'),
 (4,
  '0.056*"ethereum" + 0.018*"community" + 0.017*"blockchain" + 0.017*"project" + 0.012*"dapp" + 0.009*"build" + 0.009*"developer" + 0.008*"platform" + 0.007*"team" + 0.007*"great"'),
 (5,
  '0.171*"contract" + 0.049*"name" + 0.048*"gas" + 0.040*"address" + 0.031*"function" + 0.030*"type" + 0.029*"uint

In [343]:
two_ether_lda_dict['Q12017-Q42018'].print_topics()

[(0,
  '0.081*"token" + 0.043*"eth" + 0.025*"price" + 0.024*"exchange" + 0.022*"coin" + 0.020*"transaction" + 0.020*"pay" + 0.019*"value" + 0.018*"gas" + 0.016*"buy"'),
 (1,
  '0.078*"key" + 0.060*"lose" + 0.032*"winner" + 0.024*"private_key" + 0.020*"steal" + 0.020*"lottery" + 0.019*"output" + 0.019*"prize" + 0.017*"protect" + 0.015*"burn"'),
 (2,
  '0.017*"crypto" + 0.014*"money" + 0.014*"chain" + 0.010*"system" + 0.010*"trading" + 0.010*"market" + 0.009*"world" + 0.008*"pos" + 0.008*"proof" + 0.007*"currency"'),
 (3,
  '0.030*"ethereum" + 0.020*"would" + 0.015*"make" + 0.013*"get" + 0.013*"be" + 0.012*"not" + 0.011*"time" + 0.010*"people" + 0.010*"work" + 0.010*"good"'),
 (4,
  '0.047*"wallet" + 0.038*"eth" + 0.029*"address" + 0.026*"use" + 0.026*"send" + 0.024*"transaction" + 0.019*"account" + 0.019*"not" + 0.017*"try" + 0.013*"get"'),
 (5,
  '0.040*"amp" + 0.032*"blockchain" + 0.021*"project" + 0.017*"platform" + 0.017*"user" + 0.017*"game" + 0.012*"dapp" + 0.012*"build" + 0.011*"

In [346]:
print("Time Period & Topic \# & Bitcoin Topic & Bitcoin Topic Words & Ethereum Topic & Ethereum Topic Words", "\\\\")
dict_bit = {}
dict_ether = {}
for key, value in two_bit_lda_dict.items():
    for i in two_bit_lda_dict[key].show_topics():
        p = 0
        s = 0
        if i[0] == 0:
            index = i[0] + 1
            bit_words = re.sub("\*|\.|\d|\s|\"", "", i[1])
            bit_words = re.sub("\+", ", ", bit_words)
            count = 0
            for j in range(len(bit_words)):
                if bit_words[j] == ",":
                    count += 1
                    if dict_bit.get(bit_words[p:j], 0):
                        dict_bit[bit_words[p:j]] += 1
                    else:
                         dict_bit[bit_words[p:j]] = 1
                    p = j
                    if count == 5:
                        bit_words_1 = bit_words[0:j+1]
                        bit_words_2 = bit_words[j+2:0]
            ether_words = re.sub("\*|\.|\d|\s|\"", "", two_ether_lda_dict[key].show_topics()[i[0]][1])
            ether_words = re.sub("\+", ", ", ether_words)
            count = 0
            for k in range(len(ether_words)):
                if ether_words[k] == ",":
                    count += 1
                    if dict_ether.get(ether_words[p:j], 0):
                         dict_ether[ether_words[p:j]] += 1
                    else:
                         dict_ether[ether_words[p:j]] = 1
                    s = k
                    if count == 5:
                        ether_words_1 = ether_words[0:k+1]
                        ether_words_2 = ether_words[k+2:-1]
            print(key, "&", index, "& filler &", bit_words_1, "& filler &", ether_words_1, "\\\\")
            print("& & &", bit_words_2, "& &", ether_words_2, "\\\\")
        else:
            index = i[0] + 1
            bit_words = re.sub("\*|\.|\d|\s|\"", "", i[1])
            bit_words = re.sub("\+", ", ", bit_words)
            count = 0
            for j in range(len(bit_words)):
                if bit_words[j] == ",":
                    count += 1
                    if dict_bit.get(bit_words[p:j], 0):
                        dict_bit[bit_words[p:j]] += 1
                    else:
                         dict_bit[bit_words[p:j]] = 1
                    p = j
                    count += 1
                    if count == 5:
                        bit_words_1 = bit_words[0:j+1]
                        bit_words_2 = bit_words[j+2:-1]
            ether_words = re.sub("\*|\.|\d|\s|\"", "", two_ether_lda_dict[key].show_topics()[i[0]][1])
            ether_words = re.sub("\+", ", ", ether_words)
            count = 0
            for k in range(len(ether_words)):
                if ether_words[k] == ",":
                    count += 1
                    if dict_ether.get(ether_words[p:j], 0):
                         dict_ether[ether_words[p:j]] += 1
                    else:
                         dict_ether[ether_words[p:j]] = 1
                    s = k
                    if count == 5:
                        ether_words_1 = ether_words[0:k+1]
                        ether_words_2 = ether_words[k+2:-1]
            print("&", index, "& filler &", bit_words_1, "& filler &", ether_words_1, "\\\\")
            print("& & &", bit_words_2, "& &", ether_words_2, "\\\\")

Time Period & Topic \# & Bitcoin Topic & Bitcoin Topic Words & Ethereum Topic & Ethereum Topic Words \\
Q32015-Q42016 & 1 & filler & core, datum, hardware, digital, satoshi, & filler & would, could, make, people, think, \\
& & &  & & price, money, exchange, may, fun \\
& 2 & filler & core, datum, hardware, digital, satoshi, & filler & wallet, account, send, address, transfer, \\
& & &  & & file, key, show, web, coinbas \\
& 3 & filler & core, datum, hardware, digital, satoshi, & filler & geth, parity, sync, node, run, \\
& & &  & & client, download, version, connect, serve \\
& 4 & filler & core, datum, hardware, digital, satoshi, & filler & token, dao, investor, split, consensus, \\
& & &  & & prove, email, standard, member, rul \\
& 5 & filler & core, datum, hardware, digital, satoshi, & filler & ethereum, community, blockchain, project, dapp, \\
& & &  & & build, developer, platform, team, grea \\
& 6 & filler & core, datum, hardware, digital, satoshi, & filler & contract, name, gas

In [347]:
dict_bit

{', accept': 1,
 ', access': 2,
 ', account': 1,
 ', address': 2,
 ', advance': 1,
 ', altcoin': 1,
 ', anonymous': 1,
 ', app': 2,
 ', application': 1,
 ', arbitration': 1,
 ', backup': 1,
 ', bank': 2,
 ', bch': 1,
 ', be': 2,
 ', bet': 1,
 ', block': 1,
 ', blockchain': 1,
 ', btc': 1,
 ', btcc': 1,
 ', business': 1,
 ', buy': 2,
 ', card': 1,
 ', cent': 1,
 ', chain': 2,
 ', code': 1,
 ', coin': 1,
 ', coinbase': 1,
 ', community': 1,
 ', company': 1,
 ', computer': 1,
 ', confirm': 1,
 ', confirmation': 1,
 ', contact': 1,
 ', create': 1,
 ', credit_card': 1,
 ', crypto': 1,
 ', cryptocurrency': 1,
 ', currency': 2,
 ', datum': 1,
 ', day': 1,
 ', debit_card': 1,
 ', digital': 2,
 ', do': 2,
 ', document': 1,
 ', email': 1,
 ', euro': 1,
 ', everything': 1,
 ', exchange': 3,
 ', fee': 2,
 ', fiat': 1,
 ', full_node': 1,
 ', fund': 1,
 ', get': 2,
 ', go': 1,
 ', hardware': 2,
 ', high': 2,
 ', hour': 1,
 ', increase': 1,
 ', info': 1,
 ', join': 1,
 ', know': 2,
 ', lightning': 1,

In [348]:
dict_ether

{'': 54,
 ' bug': 9,
 ' get': 9,
 ' team, g': 9,
 ', coinb': 9,
 'ard, m': 9,
 'end': 9,
 'good': 9,
 'n, pay': 9,
 'ort, c': 9,
 'sional': 9,
 't, se': 9,
 'tect, bur': 9,
 'uest, cas': 9,
 'und, ': 9}

In [258]:
print_perplexity(bit_lda_dict, bit_text_long_dict)


Perplexity of Q12014 is: -8.279743450698229

Perplexity of Q22014 is: -8.419927886469653

Perplexity of Q32014 is: -8.322599777868152

Perplexity of Q42014 is: -8.29113366082227

Perplexity of Q12015 is: -8.291730913794312

Perplexity of Q22015 is: -8.348374439964536

Perplexity of Q32015 is: -8.248893908745323

Perplexity of Q42015 is: -8.200253912445962

Perplexity of Q12016 is: -8.2422902348793

Perplexity of Q22016 is: -8.299736076684841

Perplexity of Q32016 is: -8.131641771843432

Perplexity of Q42016 is: -8.120720771239712

Perplexity of Q12017 is: -8.100554551127127

Perplexity of Q22017 is: -8.016052575888532

Perplexity of Q32017 is: -7.99016099621643

Perplexity of Q42017 is: -7.834273882126996

Perplexity of Q12018 is: -8.065973542899282

Perplexity of Q22018 is: -8.34965928782143

Perplexity of Q32018 is: -8.491099618663153

Perplexity of Q42018 is: -8.418809417588976


In [259]:
print_perplexity(ether_lda_dict, ether_text_long_dict)


Perplexity of Q12014 is: -7.428611484143501

Perplexity of Q22014 is: -7.4667244981081815

Perplexity of Q32014 is: -7.40773008093144

Perplexity of Q42014 is: -7.241330783110095

Perplexity of Q12015 is: -7.7004037274292765

Perplexity of Q22015 is: -7.65713032362053

Perplexity of Q32015 is: -8.050683396963121

Perplexity of Q42015 is: -7.685212422013061

Perplexity of Q12016 is: -8.063917344393197

Perplexity of Q22016 is: -8.143371967408125

Perplexity of Q32016 is: -8.111004185391597

Perplexity of Q42016 is: -8.138776229130073

Perplexity of Q12017 is: -8.035456418043212

Perplexity of Q22017 is: -8.05424290871927

Perplexity of Q32017 is: -8.117462629878403

Perplexity of Q42017 is: -8.068955318278961

Perplexity of Q12018 is: -8.260643800391499

Perplexity of Q22018 is: -8.197375577909959

Perplexity of Q32018 is: -8.316479382981584

Perplexity of Q42018 is: -8.34208213040575


In [294]:
bit_lda_dict['Q32015'].print_topics()

[(0,
  '0.084*"run" + 0.052*"node" + 0.039*"core" + 0.030*"rate" + 0.028*"hardware" + 0.024*"confirmation" + 0.021*"version" + 0.019*"full_node" + 0.018*"machine" + 0.016*"banking"'),
 (1,
  '0.030*"vote" + 0.019*"revolution" + 0.015*"la" + 0.013*"human" + 0.010*"story" + 0.008*"doubt" + 0.008*"field" + 0.008*"commodity" + 0.008*"management" + 0.007*"para"'),
 (2,
  '0.065*"wallet" + 0.050*"send" + 0.029*"transaction" + 0.027*"blockchain" + 0.026*"can" + 0.024*"email" + 0.019*"device" + 0.013*"computer" + 0.012*"app" + 0.012*"number"'),
 (3,
  '0.063*"account" + 0.051*"bank" + 0.045*"exchange" + 0.035*"payment" + 0.027*"coinbase" + 0.027*"service" + 0.023*"transfer" + 0.020*"business" + 0.017*"btc" + 0.016*"deposit"'),
 (4,
  '0.015*"support" + 0.012*"link" + 0.011*"website" + 0.011*"site" + 0.011*"circle" + 0.010*"add" + 0.010*"check" + 0.009*"month" + 0.009*"info" + 0.009*"also"'),
 (5,
  '0.066*"bitcoin" + 0.029*"not" + 0.027*"would" + 0.024*"use" + 0.019*"be" + 0.018*"get" + 0.018*

In [261]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_bit = pyLDAvis.gensim.prepare(bit_lda_dict['Q12016'], bit_text_long_dict['Q12016'][0],
                                  bit_text_long_dict['Q12016'][1])
vis_bit
print('\n')

In [323]:
# Format for paper
print("Quarter & Topic \# & Bitcoin Topic & Bitcoin Topic Words & Ethereum Topic & Ethereum Topic Words", "\\\\")
for key, value in bit_lda_dict.items():
    for i in bit_lda_dict[key].show_topics():
        if i[0] == 0:
            index = i[0] + 1
            bit_words = re.sub("\*|\.|\d|\s|\"", "", i[1])
            bit_words = re.sub("\+", ", ", bit_words)
            count = 0
            for j in range(len(bit_words)):
                if bit_words[j] == ",":
                    count += 1
                    if count == 5:
                        bit_words_1 = bit_words[0:j+1]
                        bit_words_2 = bit_words[j+2:-1]
            ether_words = re.sub("\*|\.|\d|\s|\"", "", ether_lda_dict[key].show_topics()[i[0]][1])
            ether_words = re.sub("\+", ", ", ether_words)
            count = 0
            for k in range(len(ether_words)):
                if ether_words[k] == ",":
                    count += 1
                    if count == 5:
                        ether_words_1 = ether_words[0:k+1]
                        ether_words_2 = ether_words[k+2:-1]
            print(key, "&", index, "& filler &", bit_words_1, "& filler &", ether_words_1, "\\\\")
            print("& & filler &", bit_words_2, "& filler &", ether_words_2, "\\\\")
        else:
            index = i[0] + 1
            bit_words = re.sub("\*|\.|\d|\s|\"", "", i[1])
            bit_words = re.sub("\+", ", ", bit_words)
            count = 0
            for j in range(len(bit_words)):
                if bit_words[j] == ",":
                    count += 1
                    if count == 5:
                        bit_words_1 = bit_words[0:j+1]
                        bit_words_2 = bit_words[j+2:-1]
            ether_words = re.sub("\*|\.|\d|\s|\"", "", ether_lda_dict[key].show_topics()[i[0]][1])
            ether_words = re.sub("\+", ", ", ether_words)
            count = 0
            for k in range(len(ether_words)):
                if ether_words[k] == ",":
                    count += 1
                    if count == 5:
                        ether_words_1 = ether_words[0:k+1]
                        ether_words_2 = ether_words[k+2:-1]
            print("&", index, "& filler &", bit_words_1, "& filler &", ether_words_1, "\\\\")
            print("& & filler &", bit_words_2, "& filler &", ether_words_2, "\\\\")

Quarter & Topic \# & Bitcoin Topic & Bitcoin Topic Words & Ethereum Topic & Ethereum Topic Words \\
Q12014 & 1 & filler & btc, account, site, coinbase, link, & filler & transaction, contract, would, time, execute, \\
& & filler & receive, send, email, order, mtgo & filler & node, service, able, mining, bloc \\
& 2 & filler & wallet, address, transaction, use, send, & filler & inflation, value, happen, year, increase, \\
& & filler & blockchain, key, private_key, create, pape & filler & make, price, rate, would, se \\
& 3 & filler & bitcoin, tax, community, company, business, & filler & ethereum, could, would, not, be, \\
& & filler & new, accept, also, use, servic & filler & miner, make, want, mining, projec \\
& 4 & filler & app, phone, fuck, various, itune, & filler & btc, invest, founder, app, coinbase, \\
& & filler & apple, scam, recover, android, pictur & filler & hackathon, creative, ask, msc, salar \\
& 5 & filler & atm, usually, foundation, backup, evidence, & filler & ether, 

& 6 & filler & fee, somebody, cent, mempool, serve, & filler & contract, address, amount, send, btc, \\
& & filler & mbtc, analysis, complain, years_ago, rela & filler & restart, owner, play, deposit, delet \\
& 7 & filler & regulation, cheap, gemini, irs, sense, & filler & wait, hash, request, confirm, field, \\
& & filler & clean, atm, buck, political, illega & filler & trace, profitable, explanation, fork, positio \\
& 8 & filler & bank, usd, dollar, money, rate, & filler & trademark, swarm, bitcoin, foundation, probably, \\
& & filler & deposit, figure, gold, account, interes & filler & seem, credit, holiday, moderator, bran \\
& 9 & filler & import, bc, equal, alt, cad_cad, & filler & else, blockchain, website, tax, ban, \\
& & filler & cad_xƀt, xƀt_cad, canadian, symbol, disrup & filler & nothing, trust, everyone, stop, fres \\
& 10 & filler & tax, government, vote, message, news, & filler & ethereum, would, not, know, be, \\
& & filler & law, supply, guess, major, countr & fille

In [ ]:
def create_lda(sentence_dict):
    new_dict = {}
    for key, value in sentence_dict.items():
        lda_model = gensim.models.ldamodel.LdaModel(corpus=value[0],
                                           id2word=value[1],
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        new_dict[key] = lda_model
    return new_dict